# Persp 1

In [1]:
# Import packages
import pandas as pd
import plotly.express as px
import seaborn as sns

In [2]:
# Cell 8: Optional Data Analysis (Run this cell only if you want to see detailed statistics)
def run_detailed_analysis():
    """Run detailed analysis - call this function if you want to see statistics"""
    print("\n" + "="*60)
    print("📊 DETAILED DATA ANALYSIS")
    print("="*60)
    
    analyze_position_data(df)
    
    print("\n📅 YEARLY STATISTICS")
    print("="*40)
    
    yearly_stats = df.groupby('Year').agg({
        'DriverCode': 'count',
        'Grand Prix': 'nunique'
    }).rename(columns={'DriverCode': 'Total_Records', 'Grand Prix': 'Races'})
    
    # Calculate DNF rates by year
    yearly_dnf = []
    for year in sorted(df['Year'].unique()):
        year_data = df[df['Year'] == year]
        total = len(year_data)
        dnf_count = len(year_data[~year_data['FinPos'].str.isnumeric().fillna(False)])
        dnf_rate = (dnf_count / total * 100) if total > 0 else 0
        yearly_dnf.append(dnf_rate)
    
    yearly_stats['DNF_Rate'] = yearly_dnf
    
    print(yearly_stats.round(1))

# Uncomment the line below if you want to see detailed statistics:
# run_detailed_analysis()

print("\n✅ Notebook setup complete! Use the interactive dropdown above to explore different years.")# F1 Starting vs Finishing Position Heatmap
# Interactive Jupyter Notebook

# Cell 1: Install and Import Required Packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt

# Cell 2: Load and Prepare Data
def load_f1_data():
    """Load and prepare the F1 data"""
    try:
        df = pd.read_csv('merged_f1_data_1994_2022.csv')
    except FileNotFoundError:
        return None
    
    # Handle non-numeric finishing positions (DNF, DSQ, etc.)
    df['FinPos'] = df['FinPos'].astype(str)
    
    return df

# Load the data
df = load_f1_data()

# Cell 3: Data Preparation Functions
def create_heatmap_data(df, selected_year=None):
    """Create heatmap data for starting vs finishing positions"""
    
    if selected_year and selected_year != 'Total':
        df_filtered = df[df['Year'] == selected_year].copy()
        title_suffix = f" - {selected_year}"
    else:
        df_filtered = df.copy()
        title_suffix = " - All Years (1994-2022)"
    
    # Group NC, DNF, DSQ, and DQ with DNF/DSQ for display purposes
    df_filtered = df_filtered.copy()
    df_filtered['FinPos_Display'] = df_filtered['FinPos'].replace({
        'NC': 'DNF/DSQ',
        'DNF': 'DNF/DSQ', 
        'DSQ': 'DNF/DSQ',
        'DQ': 'DNF/DSQ'
    })
    
    # Get the range of starting positions (limit to reasonable range)
    max_start_pos = min(df_filtered['Pos'].max(), 26)  # F1 grid is max 26 cars
    
    # Create categories for finishing positions
    numeric_positions = []
    non_numeric_positions = []
    
    for pos in df_filtered['FinPos_Display'].unique():
        try:
            num_pos = int(float(pos))
            if 1 <= num_pos <= 30:  # Reasonable range for F1
                numeric_positions.append(num_pos)
        except (ValueError, TypeError):
            if pos != 'nan':  # Skip NaN values
                non_numeric_positions.append(pos)
    
    # Sort numeric positions
    numeric_positions = sorted(set(numeric_positions))
    
    # Define non-finisher categories with DNF/DSQ combined
    # Put DNF/DSQ first as it's most common, then other codes
    priority_codes = ['DNF/DSQ']
    other_dnf_codes = ['DNS', 'EX', 'WD']
    
    # Create ordered list of non-finisher codes
    present_priority_codes = [code for code in priority_codes if code in non_numeric_positions]
    present_other_codes = [code for code in other_dnf_codes if code in non_numeric_positions]
    remaining_codes = [code for code in non_numeric_positions 
                      if code not in priority_codes and code not in other_dnf_codes]
    
    # Create the full list of finish position categories
    finish_categories = numeric_positions + present_priority_codes + present_other_codes + sorted(remaining_codes)
    start_categories = list(range(1, max_start_pos + 1))
    
    # Create the heatmap matrix
    heatmap_data = np.zeros((len(finish_categories), len(start_categories)))
    
    # Fill the heatmap matrix
    for _, row in df_filtered.iterrows():
        start_pos = row['Pos']
        finish_pos = row['FinPos_Display']  # Use the modified display position
        
        if start_pos in start_categories:
            start_idx = start_categories.index(start_pos)
            
            # Handle finish position
            try:
                finish_pos_int = int(float(finish_pos))
                if finish_pos_int in finish_categories:
                    finish_idx = finish_categories.index(finish_pos_int)
                    heatmap_data[finish_idx, start_idx] += 1
            except (ValueError, TypeError):
                if finish_pos in finish_categories:
                    finish_idx = finish_categories.index(finish_pos)
                    heatmap_data[finish_idx, start_idx] += 1
    
    return heatmap_data, start_categories, finish_categories, title_suffix, len(df_filtered)

# Cell 4: Plotly Interactive Heatmap Function
def create_plotly_heatmap(df, selected_year=None):
    """Create an interactive heatmap using Plotly"""
    
    heatmap_data, start_categories, finish_categories, title_suffix, total_records = create_heatmap_data(df, selected_year)
    
    # Create hover text with percentages
    hover_text = []
    for i in range(len(finish_categories)):
        hover_row = []
        for j in range(len(start_categories)):
            count = int(heatmap_data[i, j])
            percentage = (count / total_records * 100) if total_records > 0 else 0
            hover_row.append(f'Starting: P{start_categories[j]}<br>Finishing: {finish_categories[i]}<br>Count: {count}<br>Percentage: {percentage:.1f}%')
        hover_text.append(hover_row)
    
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=[f"P{pos}" for pos in start_categories],
        y=[str(pos) for pos in finish_categories],
        colorscale='Viridis',
        hoverongaps=False,
        hovertemplate='%{customdata}<extra></extra>',
        customdata=hover_text,
        colorbar=dict(title="Number of Occurrences")
    ))
    
    fig.update_layout(
        title={
            'text': f'F1 Starting vs Finishing Position Heatmap{title_suffix}<br><sub>Total Records: {total_records:,}</sub>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        xaxis_title='Starting Position',
        yaxis_title='Finishing Position',
        xaxis=dict(side='bottom'),
        yaxis=dict(autorange='reversed'),  # Reverse y-axis so P1 is at top
        width=1000,
        height=700,
        font=dict(size=12)
    )
    
    return fig

# Cell 5: Interactive Widget Setup
def create_interactive_heatmap_widget(df):
    """Create an interactive widget with dropdown for year selection"""
    
    if df is None:
        return
    
    # Get available years and add 'Total' option
    years = sorted(df['Year'].unique())
    year_options = [('All Years (Total)', 'Total')] + [(str(year), year) for year in years]
    
    # Create dropdown widget
    year_dropdown = widgets.Dropdown(
        options=year_options,
        value='Total',
        description='Select Year:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px')
    )
    
    # Output widget for the plot
    plot_output = widgets.Output()
    
    def update_heatmap(change):
        """Update heatmap when dropdown changes"""
        selected_year = change['new']
        year_value = None if selected_year == 'Total' else selected_year
        
        # Update plot
        with plot_output:
            clear_output(wait=True)
            try:
                fig = create_plotly_heatmap(df, year_value)
                fig.show()
            except Exception as e:
                pass
    
    # Bind the dropdown to the update function
    year_dropdown.observe(update_heatmap, names='value')
    
    # Initial display
    with plot_output:
        try:
            fig = create_plotly_heatmap(df, None)
            fig.show()
        except Exception as e:
            pass
    
    # Create and display the widget layout
    widget_box = widgets.VBox([
        widgets.HTML("<h3>🏎️ F1 Starting vs Finishing Position Analysis</h3>"),
        year_dropdown,
        plot_output
    ])
    
    display(widget_box)

# Cell 6: Data Analysis Functions
def analyze_position_data(df):
    """Analyze interesting patterns in the data"""
    
    if df is None:
        print("❌ No data to analyze")
        return
    
    print("🔍 F1 POSITION ANALYSIS")
    print("=" * 50)
    
    # Basic statistics
    print(f"📊 Dataset Overview:")
    print(f"   • Total records: {len(df):,}")
    print(f"   • Years covered: {df['Year'].min()}-{df['Year'].max()}")
    print(f"   • Unique drivers: {df['DriverCode'].nunique()}")
    print(f"   • Unique Grand Prix: {df['Grand Prix'].nunique()}")
    
    # Finishing position breakdown with DNF/DSQ/NC grouped
    print(f"\n🏁 Finishing Position Breakdown:")
    finish_counts = df['FinPos'].value_counts()
    
    # Separate numeric and non-numeric finishes
    numeric_finishes = 0
    dnf_breakdown = {}
    
    # Group DNF, DSQ, and NC together for analysis
    dnf_dsq_nc_count = 0
    for pos, count in finish_counts.items():
        try:
            int(float(pos))
            numeric_finishes += count
        except (ValueError, TypeError):
            if pos in ['DNF', 'DSQ', 'NC']:
                dnf_dsq_nc_count += count
            else:
                dnf_breakdown[pos] = count
    
    print(f"   • Classified finishes: {numeric_finishes:,} ({numeric_finishes/len(df)*100:.1f}%)")
    
    total_non_finishes = dnf_dsq_nc_count + sum(dnf_breakdown.values())
    print(f"   • Non-finishes: {total_non_finishes:,} ({total_non_finishes/len(df)*100:.1f}%)")
    print(f"     - DNF/DSQ/NC: {dnf_dsq_nc_count:,} ({dnf_dsq_nc_count/len(df)*100:.1f}%)")
    
    for status, count in sorted(dnf_breakdown.items(), key=lambda x: x[1], reverse=True):
        print(f"     - {status}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Show individual breakdown of DNF/DSQ/NC for reference
    print(f"\n📋 DNF/DSQ/NC Individual Breakdown (for reference):")
    for code in ['DNF', 'DSQ', 'NC']:
        if code in finish_counts:
            count = finish_counts[code]
            print(f"   • {code}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Position change analysis (classified finishers only)
    df_numeric = df.copy()
    df_numeric['FinPos_Numeric'] = pd.to_numeric(df_numeric['FinPos'], errors='coerce')
    df_classified = df_numeric.dropna(subset=['FinPos_Numeric'])
    
    if len(df_classified) > 0:
        df_classified['Position_Change'] = df_classified['Pos'] - df_classified['FinPos_Numeric']
        
        print(f"\n📈 Position Changes (Classified finishers only):")
        print(f"   • Average position change: {df_classified['Position_Change'].mean():.2f}")
        print(f"   • Best comeback: +{df_classified['Position_Change'].max()} positions")
        print(f"   • Worst drop: {df_classified['Position_Change'].min()} positions")
        
        # Pole position analysis
        pole_data = df_classified[df_classified['Pos'] == 1]
        if len(pole_data) > 0:
            pole_wins = len(pole_data[pole_data['FinPos_Numeric'] == 1])
            print(f"\n🥇 Pole Position Analysis:")
            print(f"   • Pole positions: {len(pole_data):,}")
            print(f"   • Pole to win conversion: {pole_wins}/{len(pole_data)} ({pole_wins/len(pole_data)*100:.1f}%)")

# Cell 7: Create the Interactive Dashboard
print("🚀 Creating Interactive F1 Heatmap Dashboard...")
print("📝 Instructions:")
print("   1. Use the dropdown menu to select a year or 'All Years'")
print("   2. Hover over the heatmap cells to see detailed information")
print("   3. The heatmap shows how often drivers starting in position X finish in position Y")
print("   4. Darker colors indicate higher frequency")
print("")

# Create the interactive widget
create_interactive_heatmap_widget(df)


✅ Notebook setup complete! Use the interactive dropdown above to explore different years.
🚀 Creating Interactive F1 Heatmap Dashboard...
📝 Instructions:
   1. Use the dropdown menu to select a year or 'All Years'
   2. Hover over the heatmap cells to see detailed information
   3. The heatmap shows how often drivers starting in position X finish in position Y
   4. Darker colors indicate higher frequency

